# テーマ：Fashion-MNISTデータセットを畳み込みニューラルネットワーク（CNN）で学習する
## 制約
* 全体の実行時間は60分以内

## 今回参考にしたもの
* [Get started with TensorFlow 2.0 for experts](https://github.com/tensorflow/docs/blob/master/site/en/r2/tutorials/quickstart/advanced.ipynb)

## 0. Install TensorFlow 2.0

In [0]:
!pip install tensorflow-gpu==2.0.0-alpha0

## 1. Prepare dataset

In [0]:
import tensorflow_datasets as tfds
import tensorflow as tf

#print(tfds.list_builders())
dataset, info = tfds.load('fashion_mnist', as_supervised = True, with_info = True)
dataset_test, dataset_train = dataset['test'], dataset['train']
print(info)

tfds.core.DatasetInfo(
    name='fashion_mnist',
    version=1.0.0,
    description='Fashion-MNIST is a dataset of Zalando's article images consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes.',
    urls=['https://github.com/zalandoresearch/fashion-mnist'],
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10)
    },
    total_num_examples=70000,
    splits={
        'test': <tfds.core.SplitInfo num_examples=10000>,
        'train': <tfds.core.SplitInfo num_examples=60000>
    },
    supervised_keys=('image', 'label'),
    citation='"""
        @article{DBLP:journals/corr/abs-1708-07747,
          author    = {Han Xiao and
                       Kashif Rasul and
                       Roland Vollgraf},
          title     = {Fashion-MNIST: a Novel Image Dataset for Benchmark

In [0]:
def convert_types(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

batch_size = 32

dataset_train = dataset_train.map(convert_types).shuffle(10000).batch(batch_size)
dataset_test = dataset_test.map(convert_types).batch(batch_size)

## 2. Define model

In [0]:
from tensorflow.keras.layers import Conv2D, Flatten, Dense, BatchNormalization, Dropout, Activation, MaxPool2D, GlobalAveragePooling2D
from tensorflow.keras import Model

class CNNModel(Model):
    def __init__(self):
        super(CNNModel, self).__init__()
        drop_rate = 0.5
        
        self._layers = ([
            Conv2D(32, 3), # 28, 28, 1 -> 26, 26, 32
            BatchNormalization(),
            Activation(tf.nn.relu),
            Conv2D(64, 3), # 26, 26, 32 -> 24, 24, 64
            BatchNormalization(),
            Activation(tf.nn.relu),
            MaxPool2D(), # 24, 24, 64 -> 12, 12, 64
            Conv2D(128, 3), # 12, 12, 64 -> 10, 10, 128
            BatchNormalization(),
            Activation(tf.nn.relu),
            Conv2D(256, 3), # 10, 10, 128 -> 8, 8, 256
            BatchNormalization(),
            Activation(tf.nn.relu),
            MaxPool2D(), # 8, 8, 256 -> 4, 4, 256
            Flatten(), # 4, 4, 256 -> 4096
            Dense(256), # 4096 -> 256
            BatchNormalization(),
            Dropout(drop_rate),
            Activation(tf.nn.relu),
            Dense(10, activation = 'softmax') # 256 -> 10                        
        ])                
        
    def call(self, x):
        for layer in self._layers:
            x = layer(x)
        return x
       
    
model = CNNModel()

In [0]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

## 3. Prepare training

In [0]:
train_loss = tf.keras.metrics.Mean(name = 'train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'train_accuracy')

test_loss = tf.keras.metrics.Mean(name = 'test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'test_accuracy')

In [0]:
@tf.function
def train_step(image, label):
    with tf.GradientTape() as tape:
        predictions = model(image)
        loss = loss_object(label, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(label, predictions)
        
@tf.function
def test_step(image, label):
    predictions = model(image)
    loss = loss_object(label, predictions)
    
    test_loss(loss)
    test_accuracy(label, predictions)

## 4. Train

In [0]:
import time

num_epoch = 10
start_time = time.time()

for epoch in range(num_epoch):
    for image, label in dataset_train:
        train_step(image, label)
        
    for test_image, test_label in dataset_test:
        test_step(test_image, test_label)
        
    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}, spent_time: {} min'
    spent_time = time.time() - start_time
    print(template.format(epoch + 1, train_loss.result(), train_accuracy.result() * 100, test_loss.result(), test_accuracy.result() * 100, spent_time / 60))

Epoch 1, Loss: 0.4085262715816498, Accuracy: 84.93833923339844, Test Loss: 0.30398645997047424, Test Accuracy: 88.70000457763672, spent_time: 1.1197947025299073 min
Epoch 2, Loss: 0.3328215181827545, Accuracy: 87.7733383178711, Test Loss: 0.2793513834476471, Test Accuracy: 89.74000549316406, spent_time: 2.169168742497762 min
Epoch 3, Loss: 0.29196467995643616, Accuracy: 89.22777557373047, Test Loss: 0.2699553370475769, Test Accuracy: 90.25333404541016, spent_time: 3.2162970105806985 min
Epoch 4, Loss: 0.2630894184112549, Accuracy: 90.2874984741211, Test Loss: 0.2627233862876892, Test Accuracy: 90.58999633789062, spent_time: 4.259984481334686 min
Epoch 5, Loss: 0.2399316430091858, Accuracy: 91.12166595458984, Test Loss: 0.2603335380554199, Test Accuracy: 90.80599975585938, spent_time: 5.3003990888595585 min
Epoch 6, Loss: 0.22022312879562378, Accuracy: 91.83583068847656, Test Loss: 0.26244479417800903, Test Accuracy: 90.99666595458984, spent_time: 6.341218503316243 min
Epoch 7, Loss: 0.